# IPv4 Addressing

- 32 bits = 4 bytes
- address space = 2^32 addresses
- dotted decimal notation: A.B.C.D
    - A, B, C and D between 0 and 255


In [4]:
# Check if the given string is a valid IPv4 address
def is_valid(ip):
    ip = ip.split('.')
    if len(ip) != 4:
        return False
    for i in ip:
        if not i.isdigit() or not 0 <= int(i) <= 255:
            return False
    return True

is_valid('10.20.300.40')

False

In [48]:
# Convert an IPv4 address to binary
def to_binary(ip, sep=''):
    ip = ip.split('.')
    return sep.join([bin(int(i))[2:].zfill(8) for i in ip])

to_binary('127.0.0.1', ' ')

'01111111 00000000 00000000 00000001'

In [6]:
# Convert a binary IP address to decimal
def to_decimal(ip):
    # remove spaces from ip
    ip = ip.replace(' ', '')
    # split ip in 4 parts
    ip = [ip[i:i+8] for i in range(0, len(ip), 8)]
    # convert each part to decimal and join them with '.'
    return '.'.join([str(int(i, 2)) for i in ip])

to_decimal('01111111 00000000 00000000 00000001')

'127.0.0.1'

# IPv4 Classes

| Class            | Binary Prefix | Address Range               | Default Subnet Mask | Networks           | Hosts                 |
|------------------|---------------|-----------------------------|---------------------|--------------------|-----------------------|
| A                | 0             | 1.0.0.0 - 127.255.255.255   | 255.0.0.0 (/8)      | 7 bits - 128 nets  | 24 bits - 16M hosts   |
| B                | 10            | 128.0.0.0 - 191.255.255.255 | 255.255.0.0 (/16)   | 14 bits - 16k nets | 16 bits - 64k hosts   |
| C                | 110           | 192.0.0.0 - 223.255.255.255 | 255.255.255.0 (/24) | 21 bits - 2M nets  | 8 bits - 256 hosts    |
| D (multicast)    | 1110          | 224.0.0.0 - 239.255.255.255 | Not applicable      | Not applicable     | 256M multicast groups |
| E (experimental) | 1111          | 240.0.0.0 - 255.255.255.254 | Not applicable      | Not applicable     | Not applicable        |

In [7]:
# Return IPv4 class
def get_ip_class(ip):
    ip = ip.split('.')
    if 1 <= int(ip[0]) <= 127:
        return 'A'
    elif 128 <= int(ip[0]) <= 191:
        return 'B'
    elif 192 <= int(ip[0]) <= 223:
        return 'C'
    elif 224 <= int(ip[0]) <= 239:
        return 'D'
    elif 240 <= int(ip[0]) <= 255:
        return 'E'
    
get_ip_class('200.10.20.5')

'C'

# Classless addressing (CIDR - Classless Inter-Domain Routing)

- IP addresses are allocated more efficiently according to actual need
- VLSM (Variable-Length Subnet Masking) - subnet mask can be of any length
- notation - IP followed by a slash ("/") and a number indicating the number of bits in the network prefix (ex. 192.168.1.0/24)
- simplifies routing - routes can be aggregated into a single routing table entry

In [22]:
# Convert CIDR notation to IPv4 address
def cidr_to_ip(cidr):
    ip, mask = cidr.split('/')
    ip = ip.split('.')
    mask = int(mask)
    mask = '1' * mask + '0' * (32 - mask)
    mask = [mask[i:i+8] for i in range(0, 32, 8)]
    mask = [str(int(i, 2)) for i in mask]
    result = '.'.join(ip) + ' ' + '.'.join(mask)
    return result.strip()

ip = cidr_to_ip('/8')
ip + ' ' + to_binary(ip)

'255.0.0.0 11111111 00000000 00000000 00000000'

In [24]:
# Return number of addresses in a subnet given the CIDR notation
def get_num_addr(cidr):
    return 2 ** (32 - int(cidr))

get_num_addr(8)

16777216

# Reserved address

- cannot be assigned to any host

- network address - represents the entire subnet
    - network_address = IP address AND subnet mask
    - all host bits with 0
- broadcast address - used to send data to all devices on the network
    - broadcast_address = IP address OR INVERT(subnet mask)
    - all host bits with 1

In [29]:
# Return network address
def get_network_addr(ip_cidr):
    ip, mask = ip_cidr.split('/')
    ip = ip.split('.')
    mask = int(mask)
    mask = '1' * mask + '0' * (32 - mask)
    mask = [mask[i:i+8] for i in range(0, 32, 8)]
    mask = [str(int(i, 2)) for i in mask]
    network = [str(int(ip[i]) & int(mask[i])) for i in range(4)]
    return '.'.join(network)

get_network_addr('192.168.10.5/16')

'192.168.0.0'

In [30]:
# Return broadcast address
def get_broadcast_addr(ip_cidr):
    ip, mask = ip_cidr.split('/')
    ip = ip.split('.')
    mask = int(mask)
    mask = '1' * mask + '0' * (32 - mask)
    mask = [mask[i:i+8] for i in range(0, 32, 8)]
    mask = [str(int(i, 2)) for i in mask]
    broadcast = [str(int(ip[i]) | (255 - int(mask[i]))) for i in range(4)]
    return '.'.join(broadcast)

get_broadcast_addr('192.168.10.5/16')

'192.168.255.255'

In [31]:
# Return first and last usable addresses in a subnet
def get_first_last_addr(ip_cidr):
    ip, mask = ip_cidr.split('/')
    ip = ip.split('.')
    mask = int(mask)
    mask = '1' * mask + '0' * (32 - mask)
    mask = [mask[i:i+8] for i in range(0, 32, 8)]
    mask = [str(int(i, 2)) for i in mask]
    network = [str(int(ip[i]) & int(mask[i])) for i in range(4)]
    broadcast = [str(int(ip[i]) | (255 - int(mask[i]))) for i in range(4)]
    first = network.copy()
    first[-1] = str(int(first[-1]) + 1)
    last = broadcast.copy()
    last[-1] = str(int(last[-1]) - 1)
    return '.'.join(first), '.'.join(last)

get_first_last_addr('192.168.10.5/16')

('192.168.0.1', '192.168.255.254')

# Subnets

- breaks a large block of addresses allocated to an organization in smaller blocks
- creation of subnets is visible only inside the organization network - the address keep the same for outside


In [61]:
# Return the list of subnets given a network address and a number of subnets
def get_subnets(network_addr, num_subnets):
    ip, mask = network_addr.split('/')
    mask = int(mask)
    subnetbitslen = len(bin(num_subnets-1)[2:])
    newmask = mask + subnetbitslen
    subnets = []
    for i in range(num_subnets):
        binip = to_binary(ip)
        newip = binip[:mask] + bin(i)[2:].zfill(subnetbitslen) + '0' * (32 - mask - subnetbitslen)
        subnets.append(to_decimal(newip)+'/'+str(newmask))
    return subnets

get_subnets('192.168.0.0/16', 4)

['192.168.0.0/18', '192.168.64.0/18', '192.168.128.0/18', '192.168.192.0/18']

In [69]:
netaddr = '192.168.0.0/16'
numsubets = 4

print(get_num_addr(int(netaddr.split('/')[1])))
print()

for subnet in get_subnets(netaddr, numsubets):
    print(subnet)
    print(get_broadcast_addr(subnet))
    print(get_num_addr(int(subnet.split('/')[1])))
    print(get_first_last_addr(subnet))
    print()

65536

192.168.0.0/18
192.168.63.255
16384
('192.168.0.1', '192.168.63.254')

192.168.64.0/18
192.168.127.255
16384
('192.168.64.1', '192.168.127.254')

192.168.128.0/18
192.168.191.255
16384
('192.168.128.1', '192.168.191.254')

192.168.192.0/18
192.168.255.255
16384
('192.168.192.1', '192.168.255.254')

